# 🚗 ABAX Data Science Project: Overview & Introduction

**Author:** Reza Mirzaeifard
**Date:** December 2025

---

## Project Overview

This project addresses two machine learning tasks for ABAX telematics:

### Task 1: Driver Behavior Classification
**Goal:** Predict driver behavior (NORMAL, DROWSY, AGGRESSIVE) from smartphone sensor data

### Task 2: Fuel Economy Prediction
**Goal:** Predict vehicle fuel efficiency (MPG) from vehicle specifications

---

## Repository Structure

```
ABAX/
├── notebooks/
│   ├── 01_project_overview.ipynb    ← This file (project introduction)
│   ├── 02_classification.ipynb      ← Complete classification pipeline
│   ├── 03_eda_regression.ipynb      ← Regression EDA
│   ├── 04_regression.ipynb          ← Complete regression pipeline
│   └── UAH_Drive_Set.ipynb          ← Original dataset exploration
├── src/
│   ├── data/                        ← Data loading & splitting
│   ├── features/                    ← Feature engineering
│   ├── models/                      ← Model training & evaluation
│   ├── visualization/               ← Plotting utilities
│   └── utils/                       ← Helper functions
├── data/
│   ├── UAH-DRIVESET-v1/            ← Raw driving data
│   └── processed/                   ← Extracted features
├── results/
│   └── figures/                     ← Generated plots
└── docs/
    └── ABAX_Technical_Report.pdf    ← Complete technical report
```

---

## Key Results Summary

| Task | Best Model | Performance | Key Insight |
|------|------------|-------------|-------------|
| **Classification** | Logistic Regression | 75% accuracy | Simple models win on small data |
| **Regression** | Gradient Boosting | R² > 0.99 | Strong feature-target relationship |

---

## Datasets

### UAH-DriveSet (Classification)

| Attribute | Value |
|-----------|-------|
| Source | University of Alcalá |
| Drivers | 6 (D1-D6) |
| Trips | 40 total |
| Behaviors | NORMAL, DROWSY, AGGRESSIVE |
| Sensors | GPS (1Hz), Accelerometer (~50Hz) |

### EPA Fuel Economy (Regression)

| Attribute | Value |
|-----------|-------|
| Source | EPA Official Data |
| Vehicles | 5,000 (2015-2024) |
| Target | Combined MPG |
| Features | Year, cylinders, displacement, drive, class, fuel type |

---

## Key Design Decisions

### 1. Raw Features Only (No Pre-computed Scores)

**Problem:** The UAH-DriveSet includes pre-computed scores like `score_braking` and behavioral ratios like `ratio_aggressive`. Using these creates **circular logic** because they're computed using similar heuristics as the behavior labels.

**Solution:** We extract features directly from raw GPS and accelerometer data:
- Speed statistics (mean, std, max, min)
- Acceleration statistics (x, y, magnitude)
- Jerk (smoothness indicator)
- Event counts (brakes, turns, hard events)

### 2. Driver-Level Split (D6 Held Out)

**Problem:** Random splits allow the model to learn driver-specific patterns, inflating accuracy by ~20%.

**Solution:** Driver D6 is completely held out for testing. The model never sees D6 during training, simulating deployment to a new customer.

### 3. Multiple Model Comparison

We compare 16 classification algorithms and 13 regression algorithms to find the best accuracy-interpretability tradeoff.

---


In [4]:
# Quick environment check
import sys
from pathlib import Path

project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("📁 Project Structure:")
print(f"   Project root: {project_root}")

# Check key directories
dirs_to_check = ['data', 'notebooks', 'src', 'results', 'docs']
for d in dirs_to_check:
    path = project_root / d
    exists = "✅" if path.exists() else "❌"
    print(f"   {exists} {d}/")


📁 Project Structure:
   Project root: /Users/rezami/PycharmProjects/ABAX
   ✅ data/
   ✅ notebooks/
   ✅ src/
   ✅ results/
   ✅ docs/


In [5]:
# Check data files
print("\n📊 Data Files:")
data_files = [
    'data/processed/uah_raw_features.csv',
    'data/processed/epa_fuel_economy.csv',
]
for f in data_files:
    path = project_root / f
    if path.exists():
        import pandas as pd
        df = pd.read_csv(path)
        print(f"   ✅ {f}: {df.shape}")
    else:
        print(f"   ❌ {f}: NOT FOUND")



📊 Data Files:
   ✅ data/processed/uah_raw_features.csv: (40, 39)
   ✅ data/processed/epa_fuel_economy.csv: (5000, 39)


In [6]:
# Check result figures
print("\n🎨 Result Figures:")
figures = [
    'class_distribution.png',
    'classifier_comparison.png',
    'confusion_matrix_classification.png',
    'feature_importance_classification.png',
    'cnn_learning_curves_classification.png',
]
for f in figures:
    path = project_root / 'results' / 'figures' / f
    exists = "✅" if path.exists() else "❌"
    print(f"   {exists} {f}")



🎨 Result Figures:
   ✅ class_distribution.png
   ✅ classifier_comparison.png
   ✅ confusion_matrix_classification.png
   ✅ feature_importance_classification.png
   ✅ cnn_learning_curves_classification.png


---

## How to Use This Project

### 1. Classification Pipeline
Open `02_classification.ipynb` for the complete driver behavior classification:
- Raw data exploration
- Feature engineering
- EDA with visualizations
- Model training (16 algorithms)
- CNN deep learning
- Evaluation and analysis

### 2. Regression Pipeline
Open `04_regression.ipynb` for fuel economy prediction:
- Data loading and preprocessing
- Feature engineering
- Model training (13 algorithms)
- Evaluation with residual analysis

### 3. Technical Report
Read `docs/ABAX_Technical_Report.pdf` for:
- Complete methodology
- All results and figures
- Failure analysis
- Production recommendations

---

## Quick Start

```python
# Load the classification dataset
import pandas as pd
from src.data import split_by_driver

df = pd.read_csv('data/processed/uah_raw_features.csv')
X = df.drop(columns=['driver', 'behavior', 'road_type'])
y = df['behavior']

# Split with D6 held out
X_train, X_test, y_train, y_test = split_by_driver(
    df[list(X.columns) + ['driver']],
    y.values,
    test_drivers=['D6']
)

# Train best model (Logistic Regression)
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(penalty='l2', max_iter=1000)
model.fit(X_train_scaled, y_train)
accuracy = model.score(X_test_scaled, y_test)
print(f"Test Accuracy: {accuracy:.2%}")
```

---

## Key Findings

### Classification

1. **Logistic Regression wins** on this small dataset (75% accuracy)
2. **Raw sensor features** avoid circular logic from pre-computed scores
3. **D6 held-out** provides realistic evaluation for new customers
4. **Feature importance**: Speed variance, jerk, and event counts are most predictive

### Regression

1. **Ensemble methods dominate** (R² > 0.99)
2. **Vehicle class and fuel type** are most important features
3. **Linear models also strong** (R² ~ 0.85-0.90)

---

## Next Steps for Production

1. **Collect more data**: 40 trips is too small for deep learning
2. **Temporal models**: Try LSTM/Transformer on raw time-series
3. **Real-time scoring**: Implement streaming inference
4. **Driver normalization**: Adjust for individual driver baselines

---

**📚 Continue to `02_classification.ipynb` for the full classification pipeline →**
